# Example 1: Using LAMMPS with PyLammps

The LAMMPS Python package provides multiple interfaces. The `PyLammps` interface is a high-level abstration of the low-level `lammps` interface. `IPyLammps` further extends this interface with functions that are useful for Jupyter notebooks to enable embedding generated graphics and videos.

## Prerequisites

Before Running this example, make sure your Python environment can find the LAMMPS shared library (`liblammps.so`) and the LAMMPS Python package is installed. If you followed the [README](README.md) in this folder, this should already be the case. You can also find more information about how to compile LAMMPS and install the LAMMPS Python package in the [LAMMPS manual](https://docs.lammps.org/Python_install.html). There is also a dedicated [PyLammps HowTo](https://docs.lammps.org/Howto_pylammps.html).

## Creating a new simulation

Once the LAMMPS shared library and the LAMMPS Python package are installed, you can create a new LAMMMPS instance in your Python interpreter as follows:

In [ ]:
from lammps import IPyLammps
L = IPyLammps()

With `PyLammps`/`IPyLammps` you can write LAMMPS simulations similar to the input script language. Take the following LAMMPS input script:

```bash
# 3d Lennard-Jones melt

units        lj
atom_style   atomic

lattice      fcc 0.8442
region       box block 0 4 0 4 0 4
create_box   1 box
create_atoms 1 box
mass         1 1.0

velocity     all create 1.44 87287 loop geom

pair_style   lj/cut 2.5
pair_coeff   1 1 1.0 1.0 2.5

neighbor     0.3 bin
neigh_modify delay 0 every 20 check no

fix          1 all nve

thermo       50
```
The equivalent can be written with `PyLammps`/`IPyLammps`:

In [ ]:
# 3d Lennard-Jones melt

L.units("lj")
L.atom_style("atomic")

L.lattice("fcc", 0.8442)
L.region("box", "block", 0, 4, 0, 4, 0, 4)
L.create_box(1, "box")
L.create_atoms(1, "box")
L.mass(1, 1.0)

L.velocity("all", "create", 1.44, 87287, "loop geom")

L.pair_style("lj/cut", 2.5)
L.pair_coeff(1, 1, 1.0, 1.0, 2.5)

L.neighbor(0.3, "bin")
L.neigh_modify("delay", 0, "every", 20, "check no")

L.fix("1", "all", "nve")

L.thermo(50)

## Visualizing the initial state

`IPyLammps` allows you to visualize the current simulation state with the [image](https://docs.lammps.org/Python_module.html#lammps.IPyLammps.image) command. Here we use it to create an image of the initial state of the system.

In [ ]:
L.image(zoom=1.0)

## Running simulations

Use the `run` command to start the simulation. In Jupyter the return value of the last command will be displayed. The `run` command will return the output of the simulation.

In [ ]:
L.run(150)

You can suppress it by adding a semicolon `;`.

In [ ]:
L.run(100);

Visualizing the system will now show us how the atoms have moved.

In [ ]:
L.image(zoom=1.0)

## Post-processing thermo output

Independent of whether or not you suppress or show the output of the `run` command, `PyLammps` will record the output. Each `run` command creates a new entry in the `L.runs` list. So far our PyLammps instance `L` executed two `run` commands:

In [ ]:
len(L.runs)

Each entry contains information about the simulation run, including the thermo output for the printed out time steps.

```bash
# thermo output of a LAMMPS simulation run
Step Temp E_pair E_mol TotEng Press
       0         1.44   -6.7733681            0   -4.6218056   -5.0244179
      50   0.70303849   -5.6796164            0    -4.629178   0.50453907
     100   0.72628044   -5.7150774            0   -4.6299123   0.29765862
     150   0.78441711    -5.805142            0   -4.6331125 -0.086709661
```

`PyLammps` already parses this information and makes it available as dictionaries and arrays.

In [ ]:
L.runs[0]

In [ ]:
L.runs[1]

For example, the first run was 150 time steps, with printing out a line every 50 steps. You can access the list of time steps using `{entry}.thermo.Step`.

In [ ]:
L.runs[0].thermo.Step

The corresponding values of each thermo quantity are also accessed this way:

In [ ]:
L.runs[0].thermo.TotEng

Together you can use this information to run post-processing on these values or even plot it using `matplotlib`:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.xlabel('time step')
plt.ylabel('Total Energy')
plt.plot(L.runs[0].thermo.Step, L.runs[0].thermo.TotEng)